### Import packages


In [1]:
from privateAB.client import client
import torch
from privateAB.server import server_multinomial_bitflip, server_multinomial_genrr, server_ell2
from privateAB.data_generator import data_generator
import time
import numpy as np


### Change settings ################

In [7]:
sample_size = 500 # choose from {500, 1000, 1500}
n_test = 3
n_permutation = 999
significance_level = 0.05 # change continuously from .01 to .99
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Fixed settings

In [8]:
d = 500 #fixed at k=500 in the paper
privacy_level = 0.1 # fixed at alpha=0.1 in the paper

### choose between the following two settings of the probability vector:

In [9]:
p = torch.arange(1,d).float().reciprocal() #multinomial probability vector for the power law setting
#p = torch.ones(d) # multinomial probability vector for the uniform law setting
#####################################################
p = p.divide(p.sum())

### create data generator, client, and server instances

In [10]:
data_gen = data_generator() #create data generator
LDPclient = client() #create the client, which privatizes the data
server_private_vec = {
    "elltwo":server_ell2(privacy_level),
    "chi":server_multinomial_genrr(privacy_level),
    "projchi":server_multinomial_bitflip(privacy_level)
    }
server_list = ["elltwo", "chi", "projchi"]
privmech_list = ["lapu", "genrr", "bitflip"]
p_value_array = np.zeros([n_test, 3])

### Lapu + elltwo, permutation test

In [11]:

t = time.time()
for i in range(n_test):
    print(f"{i+1}th test")
    torch.manual_seed(i)
    server_private = server_private_vec["elltwo"] 
    privmech_now = "lapu"
    
    
    server_private.load_private_data_multinomial(
        LDPclient.release_private(
            privmech_now,
            data_gen.generate_multinomial_data(p, sample_size),
            d,
            privacy_level,
            device
        ),
        LDPclient.release_private(
            privmech_now,
            data_gen.generate_multinomial_data(p, sample_size),
            d,
            privacy_level,
            device
        ),
        d,
        device,
        device
        )
    p_value_array[i,0], _ = server_private.release_p_value_permutation(n_permutation)

elapsed = time.time() - t
print(elapsed)

1th test
2th test
3th test
6.006876707077026


### genrr + chi, chi-square asymptotics

In [12]:
t = time.time()
for i in range(n_test):
    print(f"{i+1}th test")
    torch.manual_seed(i)
    server_private = server_private_vec["chi"] 
    privmech_now = "genrr"
    
    
    server_private.load_private_data_multinomial(
        LDPclient.release_private(
            privmech_now,
            data_gen.generate_multinomial_data(p, sample_size),
            d,
            privacy_level,
            device
        ),
        LDPclient.release_private(
            privmech_now,
            data_gen.generate_multinomial_data(p, sample_size),
            d,
            privacy_level,
            device
        ),
        d,
        device,
        device
        )
    p_value_array[i,1], _ = server_private.release_p_value()

elapsed = time.time() - t
print(elapsed)

1th test
2th test
3th test
0.1292409896850586


### rappor + projchi, chi-square asymptotics

In [13]:
t = time.time()
for i in range(n_test):
    print(f"{i+1}th test")
    torch.manual_seed(i)
    server_private = server_private_vec["projchi"] 
    privmech_now = "bitflip"
    
    
    server_private.load_private_data_multinomial(
        LDPclient.release_private(
            privmech_now,
            data_gen.generate_multinomial_data(p, sample_size),
            d,
            privacy_level,
            device
        ),
        LDPclient.release_private(
            privmech_now,
            data_gen.generate_multinomial_data(p, sample_size),
            d,
            privacy_level,
            device
        ),
        d,
        device,
        device
        )
    p_value_array[i,2], _ = server_private.release_p_value()

elapsed = time.time() - t
print(elapsed)

1th test
2th test
3th test
0.06893038749694824


c:\Users\Jongmin\miniconda3\envs\ldptest\lib\site-packages\privateAB\server.py:271: UserWarning: torch.cholesky is deprecated in favor of torch.linalg.cholesky and will be removed in a future PyTorch release.
L = torch.cholesky(A)
should be replaced with
L = torch.linalg.cholesky(A)
and
U = torch.cholesky(A, upper=True)
should be replaced with
U = torch.linalg.cholesky(A).mH().
This transform will produce equivalent results for all valid (symmetric positive definite) inputs. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\BatchLinearAlgebra.cpp:1626.)
  self.cov_est.add(cov_est_z).div(self.n-1).to(torch.float)
